# Análisis Exploratorio de Datos - Opiniones Turísticas

Este notebook realiza un análisis exploratorio de los datos de opiniones turísticas recopilados de diferentes ciudades y atracciones en México.

## Objetivo
- Cargar y consolidar datos de todas las ciudades
- Realizar análisis básico de calidad de datos
- Explorar características generales del dataset

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración para gráficos
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

## 1. Carga y Procesamiento de Datos

Vamos a cargar todos los archivos CSV de las diferentes ciudades y crear un dataset consolidado.

In [ ]:
# Función para extraer el nombre de la atracción del nombre del archivo
def extraer_nombre_atraccion(nombre_archivo):
    """
    Extrae el nombre de la atracción del nombre del archivo CSV.
    Por ejemplo: 'cancun-la-isla.csv' -> 'la isla'
    """
    # Remover la extensión .csv
    nombre_sin_extension = nombre_archivo.replace('.csv', '')
    
    # Dividir por guiones
    partes = nombre_sin_extension.split('-')
    
    # Omitir la primera parte (que es el nombre de la ciudad)
    # y unir el resto con espacios
    if len(partes) > 1:
        atraccion = ' '.join(partes[1:])
    else:
        atraccion = nombre_sin_extension
    
    return atraccion

In [ ]:
# Función para cargar todos los datos
def cargar_datos_turisticos(ruta_data='../data'):
    """
    Carga todos los archivos CSV de las carpetas de ciudades y
    los consolida en un solo DataFrame.
    """
    dataframes = []
    
    # Obtener todas las carpetas (ciudades) en el directorio data
    carpetas_ciudades = [d for d in os.listdir(ruta_data) 
                        if os.path.isdir(os.path.join(ruta_data, d))]
    
    print(f"Ciudades encontradas: {carpetas_ciudades}")
    
    for ciudad in carpetas_ciudades:
        ruta_ciudad = os.path.join(ruta_data, ciudad)
        
        # Encontrar todos los archivos CSV en la carpeta de la ciudad
        archivos_csv = glob.glob(os.path.join(ruta_ciudad, '*.csv'))
        
        print(f"\nProcesando ciudad: {ciudad}")
        print(f"Archivos encontrados: {len(archivos_csv)}")
        
        for archivo_csv in archivos_csv:
            try:
                # Leer el archivo CSV
                df = pd.read_csv(archivo_csv)
                
                # Extraer nombre del archivo sin la ruta
                nombre_archivo = os.path.basename(archivo_csv)
                
                # Agregar columnas de ciudad y atracción
                df['ciudad'] = ciudad
                df['atraccion'] = extraer_nombre_atraccion(nombre_archivo)
                
                dataframes.append(df)
                
                print(f"  - {nombre_archivo}: {len(df)} filas")
                
            except Exception as e:
                print(f"  - Error al cargar {archivo_csv}: {e}")
    
    # Concatenar todos los DataFrames
    if dataframes:
        df_consolidado = pd.concat(dataframes, ignore_index=True)
        print(f"\n=== RESUMEN ===")
        print(f"Total de filas: {len(df_consolidado)}")
        print(f"Ciudades procesadas: {df_consolidado['ciudad'].nunique()}")
        print(f"Atracciones procesadas: {df_consolidado['atraccion'].nunique()}")
        
        return df_consolidado
    else:
        print("No se encontraron archivos CSV para procesar.")
        return pd.DataFrame()

In [ ]:
# Cargar todos los datos
df_opiniones = cargar_datos_turisticos()

In [ ]:
# Mostrar las primeras filas del dataset consolidado
print("Primeras 5 filas del dataset:")
df_opiniones.head()

## 2. Análisis Básico de Calidad de Datos

In [ ]:
# Información general del dataset
print("=== INFORMACIÓN GENERAL DEL DATASET ===")
print(f"Dimensiones: {df_opiniones.shape}")
print(f"Número de filas: {df_opiniones.shape[0]:,}")
print(f"Número de columnas: {df_opiniones.shape[1]}")
print("\nColumnas del dataset:")
for i, col in enumerate(df_opiniones.columns, 1):
    print(f"{i}. {col}")

In [ ]:
# Información detallada de tipos de datos
print("=== TIPOS DE DATOS ===")
df_opiniones.info()

In [ ]:
# Análisis de valores nulos
print("=== ANÁLISIS DE VALORES NULOS ===")
valores_nulos = df_opiniones.isnull().sum()
porcentaje_nulos = (valores_nulos / len(df_opiniones)) * 100

resumen_nulos = pd.DataFrame({
    'Valores_Nulos': valores_nulos,
    'Porcentaje': porcentaje_nulos
})

resumen_nulos = resumen_nulos.sort_values('Porcentaje', ascending=False)
print(resumen_nulos)

In [ ]:
# Análisis de duplicados
print("=== ANÁLISIS DE DUPLICADOS ===")

# Duplicados completos
duplicados_completos = df_opiniones.duplicated().sum()
print(f"Filas completamente duplicadas: {duplicados_completos}")

# Duplicados por combinación de columnas importantes
if 'Titulo' in df_opiniones.columns and 'Review' in df_opiniones.columns:
    duplicados_contenido = df_opiniones.duplicated(subset=['Titulo', 'Review', 'ciudad', 'atraccion']).sum()
    print(f"Duplicados por título + review + ciudad + atracción: {duplicados_contenido}")

# Porcentaje de duplicados
if duplicados_completos > 0:
    porcentaje_duplicados = (duplicados_completos / len(df_opiniones)) * 100
    print(f"Porcentaje de duplicados completos: {porcentaje_duplicados:.2f}%")

## 3. Análisis Descriptivo por Categorías

In [ ]:
# Distribución por ciudades
print("=== DISTRIBUCIÓN POR CIUDADES ===")
distribucion_ciudades = df_opiniones['ciudad'].value_counts()
print(distribucion_ciudades)
print(f"\nPorcentaje por ciudad:")
print((distribucion_ciudades / distribucion_ciudades.sum() * 100).round(2))

In [ ]:
# Distribución por atracciones (top 10)
print("=== TOP 10 ATRACCIONES CON MÁS OPINIONES ===")
distribucion_atracciones = df_opiniones['atraccion'].value_counts().head(10)
print(distribucion_atracciones)

In [ ]:
# Análisis de calificaciones
if 'Calificacion' in df_opiniones.columns:
    print("=== ANÁLISIS DE CALIFICACIONES ===")
    
    # Convertir calificaciones a numérico si no lo están
    df_opiniones['Calificacion'] = pd.to_numeric(df_opiniones['Calificacion'], errors='coerce')
    
    print("Estadísticas descriptivas de calificaciones:")
    print(df_opiniones['Calificacion'].describe())
    
    print("\nDistribución de calificaciones:")
    print(df_opiniones['Calificacion'].value_counts().sort_index())
    
    print(f"\nCalificación promedio general: {df_opiniones['Calificacion'].mean():.2f}")

In [ ]:
# Análisis de tipos de viaje
if 'TipoViaje' in df_opiniones.columns:
    print("=== ANÁLISIS DE TIPOS DE VIAJE ===")
    distribucion_tipos = df_opiniones['TipoViaje'].value_counts()
    print(distribucion_tipos)
    print(f"\nPorcentaje por tipo de viaje:")
    print((distribucion_tipos / distribucion_tipos.sum() * 100).round(2))

## 4. Análisis de Longitud de Textos

In [ ]:
# Análisis de longitud de títulos y reviews
if 'Titulo' in df_opiniones.columns:
    df_opiniones['longitud_titulo'] = df_opiniones['Titulo'].astype(str).str.len()
    print("=== ANÁLISIS DE LONGITUD DE TÍTULOS ===")
    print(df_opiniones['longitud_titulo'].describe())

if 'Review' in df_opiniones.columns:
    df_opiniones['longitud_review'] = df_opiniones['Review'].astype(str).str.len()
    print("\n=== ANÁLISIS DE LONGITUD DE REVIEWS ===")
    print(df_opiniones['longitud_review'].describe())
    
    # Contar palabras en reviews
    df_opiniones['palabras_review'] = df_opiniones['Review'].astype(str).str.split().str.len()
    print("\n=== NÚMERO DE PALABRAS EN REVIEWS ===")
    print(df_opiniones['palabras_review'].describe())

## 5. Análisis Temporal

In [ ]:
# Análisis temporal de opiniones
if 'FechaOpinion' in df_opiniones.columns:
    print("=== ANÁLISIS TEMPORAL DE OPINIONES ===")
    
    # Mostrar algunas fechas de ejemplo para entender el formato
    print("Ejemplos de fechas en el dataset:")
    print(df_opiniones['FechaOpinion'].head(10).tolist())
    
    # Contar valores únicos en fechas
    print(f"\nFechas únicas de opinión: {df_opiniones['FechaOpinion'].nunique()}")
    
    # Mostrar las fechas más comunes
    print("\nTop 10 fechas con más opiniones:")
    print(df_opiniones['FechaOpinion'].value_counts().head(10))

## 6. Resumen Ejecutivo

In [ ]:
# Resumen ejecutivo del análisis
print("\n" + "="*60)
print("                    RESUMEN EJECUTIVO")
print("="*60)

print(f"📊 VOLUMEN DE DATOS:")
print(f"   • Total de opiniones: {len(df_opiniones):,}")
print(f"   • Ciudades analizadas: {df_opiniones['ciudad'].nunique()}")
print(f"   • Atracciones totales: {df_opiniones['atraccion'].nunique()}")

print(f"\n🏙️ DISTRIBUCIÓN POR CIUDADES:")
for ciudad, cantidad in df_opiniones['ciudad'].value_counts().items():
    porcentaje = (cantidad / len(df_opiniones)) * 100
    print(f"   • {ciudad.upper()}: {cantidad:,} opiniones ({porcentaje:.1f}%)")

if 'Calificacion' in df_opiniones.columns:
    print(f"\n⭐ CALIFICACIONES:")
    print(f"   • Promedio general: {df_opiniones['Calificacion'].mean():.2f}/5")
    print(f"   • Mediana: {df_opiniones['Calificacion'].median():.1f}/5")

if 'TipoViaje' in df_opiniones.columns:
    tipo_mas_comun = df_opiniones['TipoViaje'].mode()[0]
    print(f"\n👥 TIPO DE VIAJE:")
    print(f"   • Más común: {tipo_mas_comun}")

print(f"\n🔍 CALIDAD DE DATOS:")
print(f"   • Duplicados completos: {df_opiniones.duplicated().sum()}")
if valores_nulos.sum() > 0:
    print(f"   • Campos con valores nulos: {(valores_nulos > 0).sum()}")
else:
    print(f"   • Sin valores nulos detectados")

print("\n✅ Dataset listo para análisis más profundos!")
print("="*60)

In [ ]:
# Guardar el dataset consolidado (opcional)
# df_opiniones.to_csv('../data/dataset_consolidado_opiniones.csv', index=False)
# print("Dataset consolidado guardado como 'dataset_consolidado_opiniones.csv'")